# Alpaca-py options trading basic

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/alpacahq/alpaca-py/blob/master/examples/options/options-trading-basic.ipynb)

- This notebook shows how to use alpaca-py with options trading API endpoints
- Please use ``paper account``. Please ``DO NOT`` use this notebook with live account. In this notebook, we place orders for options as an example.

In [57]:
# Please change the following to your own PAPER api key and secret
# or set them as environment variables (ALPACA_API_KEY, ALPACA_SECRET_KEY).
# You can get them from https://alpaca.markets/

api_key = "PK2J1QPDWL4V4AZ7GSDJ"
secret_key = "sSoyyEurTAgs8BoOcxbOBoEmDbX5yHx1Qse6PrBJ"

#### We use paper environment for this example ####
paper = True # Please do not modify this. This example is for paper trading only.
####

# Below are the variables for development this documents
# Please do not change these variables

trade_api_url = None
trade_api_wss = None
data_api_url = None
option_stream_data_wss = None

In [58]:
import os

if api_key is None:
    api_key = os.environ.get('ALPACA_API_KEY')

if secret_key is None:
    secret_key = os.environ.get('ALPACA_SECRET_KEY')

In [59]:
# install alpaca-py if it is not available
try:
    import alpaca
except ImportError:
    !python3 -m pip install alpaca-py
    import alpaca

In [60]:
import json
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo

from alpaca.trading.client import TradingClient
from alpaca.data.timeframe import TimeFrame, TimeFrameUnit
from alpaca.data.historical.option import OptionHistoricalDataClient
from alpaca.trading.stream import TradingStream
from alpaca.data.live.option import OptionDataStream

from alpaca.data.requests import (
    OptionBarsRequest,
    OptionTradesRequest,
    OptionLatestQuoteRequest,
    OptionLatestTradeRequest,
    OptionSnapshotRequest,
    OptionChainRequest
)
from alpaca.trading.requests import (
    GetOptionContractsRequest,
    GetAssetsRequest,
    MarketOrderRequest,
    GetOrdersRequest,
    ClosePositionRequest
)
from alpaca.trading.enums import (
    AssetStatus,
    ExerciseStyle,
    OrderSide,
    OrderType,
    TimeInForce,
    QueryOrderStatus
)
from alpaca.common.exceptions import APIError

In [61]:
# to run async code in jupyter notebook
import nest_asyncio
nest_asyncio.apply()

In [62]:
# check version of alpaca-py
alpaca.__version__

'0.42.0'

# Trading Client

In [63]:
# setup clients
trade_client = TradingClient(api_key=api_key, secret_key=secret_key, paper=paper, url_override=trade_api_url)

In [64]:
# check trading account
# There are trhee new columns in the account object:
# - options_buying_power
# - options_approved_level
# - options_trading_level
acct = trade_client.get_account()
acct

{   'account_blocked': False,
    'account_number': 'PA3EYHMQ636L',
    'accrued_fees': '0',
    'buying_power': '199985.88',
    'cash': '99992.94',
    'created_at': datetime.datetime(2025, 7, 11, 16, 17, 14, 187954, tzinfo=TzInfo(UTC)),
    'crypto_status': <AccountStatus.ACTIVE: 'ACTIVE'>,
    'currency': 'USD',
    'daytrade_count': 1,
    'daytrading_buying_power': '0',
    'equity': '99997.94',
    'id': UUID('f87d492e-df21-4004-bc91-f60d774c94f5'),
    'initial_margin': '0',
    'last_equity': '100000',
    'last_maintenance_margin': '0',
    'long_market_value': '5',
    'maintenance_margin': '0',
    'multiplier': '2',
    'non_marginable_buying_power': '99992.94',
    'options_approved_level': 3,
    'options_buying_power': '99992.94',
    'options_trading_level': 3,
    'pattern_day_trader': False,
    'pending_transfer_in': None,
    'pending_transfer_out': None,
    'portfolio_value': '99997.94',
    'regt_buying_power': '199985.88',
    'short_market_value': '0',
    'sh

In [65]:
# check account configuration
# - we have new field `max_options_trading_level`
acct_config = trade_client.get_account_configurations()
acct_config

{   'dtbp_check': <DTBPCheck.ENTRY: 'entry'>,
    'fractional_trading': True,
    'max_margin_multiplier': '4',
    'max_options_trading_level': None,
    'no_shorting': False,
    'pdt_check': <PDTCheck.ENTRY: 'entry'>,
    'ptp_no_exception_entry': False,
    'suspend_trade': False,
    'trade_confirm_email': <TradeConfirmationEmail.ALL: 'all'>}

In [66]:
# get list of assets which are options enabled
# - we can filter assets by `options_enabled` attribute
# - asset object has `options_enabled` attribute if it is options enabled
req = GetAssetsRequest(
  attributes = "options_enabled"
)
assets = trade_client.get_all_assets(req)
assets[:2]

[{   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
     'attributes': ['fractional_eh_enabled', 'has_options'],
     'easy_to_borrow': False,
     'exchange': <AssetExchange.OTC: 'OTC'>,
     'fractionable': True,
     'id': UUID('fffd740c-a686-4fa2-87a6-545d7da617b4'),
     'maintenance_margin_requirement': 100.0,
     'marginable': False,
     'min_order_size': None,
     'min_trade_increment': None,
     'name': 'NIKOLA CORP COM NEW',
     'price_increment': None,
     'shortable': False,
     'status': <AssetStatus.ACTIVE: 'active'>,
     'symbol': 'NKLAQ',
     'tradable': False},
 {   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
     'attributes': ['has_options'],
     'easy_to_borrow': False,
     'exchange': <AssetExchange.OTC: 'OTC'>,
     'fractionable': False,
     'id': UUID('7ed25e8e-ab7e-4c5d-ba9f-8637a7e5f54f'),
     'maintenance_margin_requirement': 100.0,
     'marginable': False,
     'min_order_size': None,
     'min_trade_increment': None,
     'name': 

In [67]:
# get list of options contracts for the given underlying symbol (e.g. SPY,AAPL)
# - get_option_contracts() is a new method to get list of options contracts
# - in this example, we get 2 options contracts for SPY,AAPL
# - you can continue to fetch options contracts by specifying page_token from next_page_token of response
underlying_symbols = ["SPY", "AAPL"]
req = GetOptionContractsRequest(
    underlying_symbols = underlying_symbols,               # specify underlying symbols
    status = AssetStatus.ACTIVE,                           # specify asset status: active (default)
    expiration_date = None,                                # specify expiration date (specified date + 1 day range)
    expiration_date_gte = None,                            # we can pass date object
    expiration_date_lte = None,                            # or string (YYYY-MM-DD)
    root_symbol = None,                                    # specify root symbol
    type = None,                                           # specify option type (ContractType.CALL or ContractType.PUT)
    style = None,                                          # specify option style (ContractStyle.AMERICAN or ContractStyle.EUROPEAN)
    strike_price_gte = None,                               # specify strike price range
    strike_price_lte = None,                               # specify strike price range
    limit = 2,                                             # specify limit
    page_token = None,                                     # specify page token
)
res = trade_client.get_option_contracts(req)
res

{   'next_page_token': 'Mg==',
    'option_contracts': [   {   'close_price': '102.9',
                                'close_price_date': datetime.date(2025, 7, 10),
                                'expiration_date': datetime.date(2025, 7, 11),
                                'id': '93403446-9372-4f2e-a589-7bf9f95547b1',
                                'name': 'AAPL Jul 11 2025 110 Call',
                                'open_interest': '130',
                                'open_interest_date': datetime.date(2025, 7, 9),
                                'root_symbol': 'AAPL',
                                'size': '100',
                                'status': <AssetStatus.ACTIVE: 'active'>,
                                'strike_price': 110.0,
                                'style': <ExerciseStyle.AMERICAN: 'american'>,
                                'symbol': 'AAPL250711C00110000',
                                'tradable': True,
                                'type': <Cont

In [68]:
# continue to fetch option contracts if there is next_page_token in response
if res.next_page_token is not None:
    req = GetOptionContractsRequest(
        underlying_symbols = underlying_symbols,               # specify underlying symbols
        status = AssetStatus.ACTIVE,                           # specify asset status: active (default)
        expiration_date = None,                                # specify expiration date (specified date + 1 day range)
        expiration_date_gte = None,                            # we can pass date object
        expiration_date_lte = None,                            # or string (YYYY-MM-DD)
        root_symbol = None,                                    # specify root symbol
        type = None,                                           # specify option type (ContractType.CALL or ContractType.PUT)
        style = None,                                          # specify option style (ContractStyle.AMERICAN or ContractStyle.EUROPEAN)
        strike_price_gte = None,                               # specify strike price range
        strike_price_lte = None,                               # specify strike price range
        limit = 2,                                             # specify limit
        page_token = res.next_page_token,                      # specify page token
    )
    res = trade_client.get_option_contracts(req)
    display(res)

{   'next_page_token': 'NA==',
    'option_contracts': [   {   'close_price': None,
                                'close_price_date': None,
                                'expiration_date': datetime.date(2025, 7, 11),
                                'id': '902cd67e-bbe8-4fb7-a0cb-05c7bea5caae',
                                'name': 'AAPL Jul 11 2025 125 Call',
                                'open_interest': None,
                                'open_interest_date': None,
                                'root_symbol': 'AAPL',
                                'size': '100',
                                'status': <AssetStatus.ACTIVE: 'active'>,
                                'strike_price': 125.0,
                                'style': <ExerciseStyle.AMERICAN: 'american'>,
                                'symbol': 'AAPL250711C00125000',
                                'tradable': True,
                                'type': <ContractType.CALL: 'call'>,
                       

In [69]:
# get options contract by symbol
# - get_option_contract() is a new method to get options contract by symbol or id
symbol = res.option_contracts[0].symbol
contract = trade_client.get_option_contract(symbol)
contract

{   'close_price': None,
    'close_price_date': None,
    'expiration_date': datetime.date(2025, 7, 11),
    'id': '902cd67e-bbe8-4fb7-a0cb-05c7bea5caae',
    'name': 'AAPL Jul 11 2025 125 Call',
    'open_interest': None,
    'open_interest_date': None,
    'root_symbol': 'AAPL',
    'size': '100',
    'status': <AssetStatus.ACTIVE: 'active'>,
    'strike_price': 125.0,
    'style': <ExerciseStyle.AMERICAN: 'american'>,
    'symbol': 'AAPL250711C00125000',
    'tradable': True,
    'type': <ContractType.CALL: 'call'>,
    'underlying_asset_id': UUID('b0b6dd9d-8b9b-48a9-ba46-b9d54906e415'),
    'underlying_symbol': 'AAPL'}

In [70]:
# get options contract by id
id = res.option_contracts[0].id
contract = trade_client.get_option_contract(symbol_or_id=id)
contract

{   'close_price': None,
    'close_price_date': None,
    'expiration_date': datetime.date(2025, 7, 11),
    'id': '902cd67e-bbe8-4fb7-a0cb-05c7bea5caae',
    'name': 'AAPL Jul 11 2025 125 Call',
    'open_interest': None,
    'open_interest_date': None,
    'root_symbol': 'AAPL',
    'size': '100',
    'status': <AssetStatus.ACTIVE: 'active'>,
    'strike_price': 125.0,
    'style': <ExerciseStyle.AMERICAN: 'american'>,
    'symbol': 'AAPL250711C00125000',
    'tradable': True,
    'type': <ContractType.CALL: 'call'>,
    'underlying_asset_id': UUID('b0b6dd9d-8b9b-48a9-ba46-b9d54906e415'),
    'underlying_symbol': 'AAPL'}

In [71]:
# get put options contracts
underlying_symbols = ["SPY"]

# specify expiration date range
now = datetime.now(tz = ZoneInfo("America/New_York"))
day1 = now + timedelta(days = 1)
day60 = now + timedelta(days = 60)

req = GetOptionContractsRequest(
    underlying_symbols = underlying_symbols,                     # specify underlying symbols
    status = AssetStatus.ACTIVE,                                 # specify asset status: active (default)
    expiration_date = None,                                      # specify expiration date (specified date + 1 day range)
    expiration_date_gte = day1.date(),                           # we can pass date object
    expiration_date_lte = day60.strftime(format = "%Y-%m-%d"),   # or string
    root_symbol = None,                                          # specify root symbol
    type = "put",                                                # specify option type: put
    style = ExerciseStyle.AMERICAN,                              # specify option style: american
    strike_price_gte = None,                                     # specify strike price range
    strike_price_lte = None,                                     # specify strike price range
    limit = 100,                                                 # specify limit
    page_token = None,                                           # specify page
)
res = trade_client.get_option_contracts(req)
res.option_contracts[:2]

[{   'close_price': '0.02',
     'close_price_date': datetime.date(2025, 7, 3),
     'expiration_date': datetime.date(2025, 7, 14),
     'id': '9e2c0b07-be2e-4a31-b449-8a92a0c56381',
     'name': 'SPY Jul 14 2025 450 Put',
     'open_interest': '7',
     'open_interest_date': datetime.date(2025, 7, 9),
     'root_symbol': 'SPY',
     'size': '100',
     'status': <AssetStatus.ACTIVE: 'active'>,
     'strike_price': 450.0,
     'style': <ExerciseStyle.AMERICAN: 'american'>,
     'symbol': 'SPY250714P00450000',
     'tradable': True,
     'type': <ContractType.PUT: 'put'>,
     'underlying_asset_id': UUID('b28f4066-5c6d-479b-a2af-85dc1a8f16fb'),
     'underlying_symbol': 'SPY'},
 {   'close_price': '0.01',
     'close_price_date': datetime.date(2025, 7, 3),
     'expiration_date': datetime.date(2025, 7, 14),
     'id': '0ccb86be-c86b-4b1b-8dfa-ab5e56eb7e74',
     'name': 'SPY Jul 14 2025 455 Put',
     'open_interest': '314',
     'open_interest_date': datetime.date(2025, 7, 9),
     'ro

In [72]:
# get high open_interest contract
open_interest = 0
high_open_interest_contract = None
for contract in res.option_contracts:
    if (contract.open_interest is not None) and (int(contract.open_interest) > open_interest):
        open_interest = int(contract.open_interest)
        high_open_interest_contract = contract
high_open_interest_contract

{   'close_price': '0.05',
    'close_price_date': datetime.date(2025, 7, 10),
    'expiration_date': datetime.date(2025, 7, 14),
    'id': '9f2f4b36-06a1-4a8f-9e5e-d82dd317b66d',
    'name': 'SPY Jul 14 2025 605 Put',
    'open_interest': '9265',
    'open_interest_date': datetime.date(2025, 7, 9),
    'root_symbol': 'SPY',
    'size': '100',
    'status': <AssetStatus.ACTIVE: 'active'>,
    'strike_price': 605.0,
    'style': <ExerciseStyle.AMERICAN: 'american'>,
    'symbol': 'SPY250714P00605000',
    'tradable': True,
    'type': <ContractType.PUT: 'put'>,
    'underlying_asset_id': UUID('b28f4066-5c6d-479b-a2af-85dc1a8f16fb'),
    'underlying_symbol': 'SPY'}

In [73]:
# place buy put option order
# - we can place buy put option order same as buy stock/crypto order
req = MarketOrderRequest(
    symbol = high_open_interest_contract.symbol,
    qty = 1,
    side = OrderSide.BUY,
    type = OrderType.MARKET,
    time_in_force = TimeInForce.DAY,
)
res = trade_client.submit_order(req)
res

{   'asset_class': <AssetClass.US_OPTION: 'us_option'>,
    'asset_id': UUID('9f2f4b36-06a1-4a8f-9e5e-d82dd317b66d'),
    'canceled_at': None,
    'client_order_id': '92a037a8-4412-423e-aee9-81df54db1b6e',
    'created_at': datetime.datetime(2025, 7, 11, 18, 33, 18, 313612, tzinfo=TzInfo(UTC)),
    'expired_at': None,
    'expires_at': datetime.datetime(2025, 7, 11, 20, 15, tzinfo=TzInfo(UTC)),
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': None,
    'id': UUID('b51bda15-d624-4ff8-b5e7-1785a2678e4b'),
    'legs': None,
    'limit_price': None,
    'notional': None,
    'order_class': <OrderClass.SIMPLE: 'simple'>,
    'order_type': <OrderType.MARKET: 'market'>,
    'position_intent': <PositionIntent.BUY_TO_OPEN: 'buy_to_open'>,
    'qty': '1',
    'ratio_qty': None,
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': <OrderSide.BUY: 'buy'>,
    'status': <OrderSta

In [74]:
# get list of orders by specifying option contract symbol
req = GetOrdersRequest(
    status = QueryOrderStatus.ALL,
    symbols = [high_open_interest_contract.symbol],
    limit = 2,
)
orders = trade_client.get_orders(req)
orders

[{   'asset_class': <AssetClass.US_OPTION: 'us_option'>,
     'asset_id': UUID('9f2f4b36-06a1-4a8f-9e5e-d82dd317b66d'),
     'canceled_at': None,
     'client_order_id': '92a037a8-4412-423e-aee9-81df54db1b6e',
     'created_at': datetime.datetime(2025, 7, 11, 18, 33, 18, 313612, tzinfo=TzInfo(UTC)),
     'expired_at': None,
     'expires_at': datetime.datetime(2025, 7, 11, 20, 15, tzinfo=TzInfo(UTC)),
     'extended_hours': False,
     'failed_at': None,
     'filled_at': None,
     'filled_avg_price': None,
     'filled_qty': '0',
     'hwm': None,
     'id': UUID('b51bda15-d624-4ff8-b5e7-1785a2678e4b'),
     'legs': None,
     'limit_price': None,
     'notional': None,
     'order_class': <OrderClass.SIMPLE: 'simple'>,
     'order_type': <OrderType.MARKET: 'market'>,
     'position_intent': <PositionIntent.BUY_TO_OPEN: 'buy_to_open'>,
     'qty': '1',
     'ratio_qty': None,
     'replaced_at': None,
     'replaced_by': None,
     'replaces': None,
     'side': <OrderSide.BUY: 'buy'

In [75]:
# below cells should be done after market open otherwise there is no position for the option contract

# get positions filtered by option contract symbol
# if you do this example outside of market hours, you will see empty list
# because we have no position in this option contract
# please wait market open and run this example again
positions = trade_client.get_all_positions()
[pos for pos in positions if pos.symbol == high_open_interest_contract.symbol]

[{   'asset_class': <AssetClass.US_OPTION: 'us_option'>,
     'asset_id': UUID('9f2f4b36-06a1-4a8f-9e5e-d82dd317b66d'),
     'asset_marginable': True,
     'avg_entry_price': '0.06',
     'avg_entry_swap_rate': None,
     'change_today': '0',
     'cost_basis': '6',
     'current_price': '0.05',
     'exchange': <AssetExchange.EMPTY: ''>,
     'lastday_price': '0.05',
     'market_value': '5',
     'qty': '1',
     'qty_available': '1',
     'side': <PositionSide.LONG: 'long'>,
     'swap_rate': None,
     'symbol': 'SPY250714P00605000',
     'unrealized_intraday_pl': '-1',
     'unrealized_intraday_plpc': '-0.1666666666666667',
     'unrealized_pl': '-1',
     'unrealized_plpc': '-0.1666666666666667',
     'usd': None}]

In [76]:
# get positions by symbol
trade_client.get_open_position(symbol_or_asset_id=high_open_interest_contract.symbol)


{   'asset_class': <AssetClass.US_OPTION: 'us_option'>,
    'asset_id': UUID('9f2f4b36-06a1-4a8f-9e5e-d82dd317b66d'),
    'asset_marginable': True,
    'avg_entry_price': '0.06',
    'avg_entry_swap_rate': None,
    'change_today': '0',
    'cost_basis': '6',
    'current_price': '0.05',
    'exchange': <AssetExchange.EMPTY: ''>,
    'lastday_price': '0.05',
    'market_value': '5',
    'qty': '1',
    'qty_available': '1',
    'side': <PositionSide.LONG: 'long'>,
    'swap_rate': None,
    'symbol': 'SPY250714P00605000',
    'unrealized_intraday_pl': '-1',
    'unrealized_intraday_plpc': '-0.1666666666666667',
    'unrealized_pl': '-1',
    'unrealized_plpc': '-0.1666666666666667',
    'usd': None}

In [77]:
# get positions by contract id
trade_client.get_open_position(symbol_or_asset_id = high_open_interest_contract.id)

{   'asset_class': <AssetClass.US_OPTION: 'us_option'>,
    'asset_id': UUID('9f2f4b36-06a1-4a8f-9e5e-d82dd317b66d'),
    'asset_marginable': True,
    'avg_entry_price': '0.06',
    'avg_entry_swap_rate': None,
    'change_today': '0',
    'cost_basis': '12',
    'current_price': '0.05',
    'exchange': <AssetExchange.EMPTY: ''>,
    'lastday_price': '0.05',
    'market_value': '10',
    'qty': '2',
    'qty_available': '2',
    'side': <PositionSide.LONG: 'long'>,
    'swap_rate': None,
    'symbol': 'SPY250714P00605000',
    'unrealized_intraday_pl': '-2',
    'unrealized_intraday_plpc': '-0.1666666666666667',
    'unrealized_pl': '-2',
    'unrealized_plpc': '-0.1666666666666667',
    'usd': None}

In [78]:
# close the option position
trade_client.close_position(
    symbol_or_asset_id = high_open_interest_contract.symbol,
    close_options = ClosePositionRequest(qty = "1")
)

{   'asset_class': <AssetClass.US_OPTION: 'us_option'>,
    'asset_id': UUID('9f2f4b36-06a1-4a8f-9e5e-d82dd317b66d'),
    'canceled_at': None,
    'client_order_id': '62d3b885-2b56-4a03-97b3-4303d1862ff5',
    'created_at': datetime.datetime(2025, 7, 11, 18, 33, 18, 469342, tzinfo=TzInfo(UTC)),
    'expired_at': None,
    'expires_at': datetime.datetime(2025, 7, 11, 20, 15, tzinfo=TzInfo(UTC)),
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': None,
    'id': UUID('d6c5ab18-b178-4320-a8e8-1260e39f93a8'),
    'legs': None,
    'limit_price': None,
    'notional': None,
    'order_class': <OrderClass.SIMPLE: 'simple'>,
    'order_type': <OrderType.MARKET: 'market'>,
    'position_intent': <PositionIntent.SELL_TO_CLOSE: 'sell_to_close'>,
    'qty': '1',
    'ratio_qty': None,
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': <OrderSide.SELL: 'sell'>,
    'status': <Or

In [79]:
# exercise the options position
# - this method does not return anything
trade_client.exercise_options_position(
    symbol_or_contract_id = high_open_interest_contract.symbol
)

APIError: {"code":40310000,"message":"insufficient underlying qty available for deliverable: SPY (requested: 100, available: 0)"}

# Trade Update (Stream)

With TradingStream client, you can get updates about trades

fyi. you can open this notebook in another window and run below cell to check trade updates.

In [ ]:
# subscribe trade updates
trade_stream_client = TradingStream(api_key, secret_key, paper=paper, url_override = trade_api_wss)

async def trade_updates_handler(data):
    print(data)

trade_stream_client.subscribe_trade_updates(trade_updates_handler)
trade_stream_client.run()

# Market Data (Historical)

In [ ]:
# setup option historical data client
option_historical_data_client = OptionHistoricalDataClient(api_key, secret_key, url_override = data_api_url)

In [ ]:
# get options historical bars by symbol
req = OptionBarsRequest(
    symbol_or_symbols = high_open_interest_contract.symbol,
    timeframe = TimeFrame(amount = 1, unit = TimeFrameUnit.Hour),   # specify timeframe
    start = now - timedelta(days = 5),                              # specify start datetime, default=the beginning of the current day.
    # end_date=None,                                                # specify end datetime, default=now
    limit = 2,                                                      # specify limit
)
option_historical_data_client.get_option_bars(req).df

In [ ]:
# get options historical trades by symbol
req = OptionTradesRequest(
    symbol_or_symbols = high_open_interest_contract.symbol,
    start = now - timedelta(days = 5),                              # specify start datetime, default=the beginning of the current day.
    # end=None,                                                     # specify end datetime, default=now
    limit = 2,                                                      # specify limit
)
option_historical_data_client.get_option_trades(req).df

In [ ]:
# get options exchange codes
option_historical_data_client.get_option_exchange_codes()

In [ ]:
# get option latest quote by symbol
req = OptionLatestQuoteRequest(
    symbol_or_symbols = [high_open_interest_contract.symbol],
)
option_historical_data_client.get_option_latest_quote(req)

In [ ]:
# get option latest trade by symbol
req = OptionLatestTradeRequest(
    symbol_or_symbols = [high_open_interest_contract.symbol],
)
option_historical_data_client.get_option_latest_trade(req)

In [ ]:
# get option snapshot by symbol
req = OptionSnapshotRequest(
    symbol_or_symbols = [high_open_interest_contract.symbol],
)
option_historical_data_client.get_option_snapshot(req)

In [ ]:
# get option chain by underlying_symbol
req = OptionChainRequest(
    underlying_symbol = high_open_interest_contract.underlying_symbol,
)
option_historical_data_client.get_option_chain(req)

# Market Data (Stream)

In [ ]:
option_data_stream_client = OptionDataStream(api_key, secret_key, url_override = option_stream_data_wss)

async def option_data_stream_handler(data):
    print(data)

symbols = [
    high_open_interest_contract.symbol,
]

option_data_stream_client.subscribe_quotes(option_data_stream_handler, *symbols)
option_data_stream_client.subscribe_trades(option_data_stream_handler, *symbols)

option_data_stream_client.run()